# Cache in Python

## Functools

`@functools.lru_cache(maxsize=128, typed=False)`

If *`maxsize`* is set to `None`, the LRU feature is disabled and the cache can grow without bound.

If *`typed`* is set to True, function arguments of different types will be cached separately. For example, `f(3)` and `f(3.0)` will be treated as distinct calls with distinct results.

In [14]:
# Example:
from functools import lru_cache

@lru_cache(maxsize=2)
def sum_(a,b):
    print(f'compute {a} + {b}')
    return a+b

print(sum_(1,2))
print(sum_(1,2))
print(sum_(2,2))
print(sum_(2,3))
print(sum_(1,2))

compute 1 + 2
3
3
compute 2 + 2
4
compute 2 + 3
5
compute 1 + 2
3


### cache with ttl (time to live) 

In [36]:
from functools import lru_cache, wraps
from datetime import datetime, timedelta
import time

def timed_lru_cache(seconds: int, maxsize: int = 128):
    def wrapper_cache(func):
        func = lru_cache(maxsize=maxsize)(func)
        func.lifetime = timedelta(seconds=seconds)
        func.expiration = datetime.utcnow() + func.lifetime

        @wraps(func)
        def wrapped_func(*args, **kwargs):
            if datetime.utcnow() >= func.expiration:
                func.cache_clear()
                func.expiration = datetime.utcnow() + func.lifetime

            return func(*args, **kwargs)

        return wrapped_func

    return wrapper_cache

@timed_lru_cache(5)
def product(a,b):
    print(f'Calling product: {a} * {b}:')
    return a*b

print(product(1,2))
print(product(1,2))
time.sleep(5)
print(product(1,2))

Calling product: 1 * 2:
2
2
Calling product: 1 * 2:
2


## cachetools

This module provides various memoizing collections and decorators, including variants of the Python Standard Library’s `@lru_cache` function decorator.

In [13]:
from cachetools.func import lru_cache

@lru_cache(maxsize=2)
def sum_(a,b):
    print(f'compute {a} + {b}')
    return a+b

print(sum_(1,2))
print(sum_(1,2))
print(sum_(2,2))
print(sum_(2,3))
print(sum_(1,2))

compute 1 + 2
3
3
compute 2 + 2
4
compute 2 + 3
5
compute 1 + 2
3


### cache with ttl (time to live)

In [18]:
import time

from cachetools import cached, TTLCache

# based on a Least Recently Used (LRU) algorithm with a per-item time-to-live (TTL) value.
@cached(cache=TTLCache(maxsize=5,ttl=5))
def product(a,b):
    print(f'Calling product: {a} * {b}:')
    return a*b

print(product(1,2))
print(product(1,2))
time.sleep(5)
print(product(1,2))

Calling product: 1 * 2:
2
2
Calling product: 1 * 2:
2


ref:

- https://docs.python.org/3/library/functools.html
- https://realpython.com/lru-cache-python/
- https://github.com/tkem/cachetools/tree/master
- https://cachetools.readthedocs.io/en/stable/#